# Efficacité

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from uncertainties import unumpy as unp
from uncertainties import ufloat, umath
from options import *
from constants import *

mpl.rcParams.update(rcParams)

In [2]:
def load(filename: str):
    return pd.read_csv(filename)

In [3]:
phi2 = U_fil * I_fil
print(f"Phi 2: {phi2}")

Phi 2: 171.92+/-0.19


In [4]:
print(f"V_max = {V_max:.1uP} cm^3")
print(f"V_min = {V_min:.1uP} cm^3")

V_max = (3.7±0.3)×10² cm^3
V_min = (7±1)×10¹ cm^3


## Theorique

In [5]:
rho_theo = 1 - (T_eau_in + T_eau_out) / (2 * T_fil)
print(f"Efficacité théorique: {rho_theo:.1u%}")

Efficacité théorique: (74+/-2)%


## Diagramme PV

In [15]:
x_density = (V_max - V_min) * 1e-6 / 211 # m^3 / bloc
y_density = (P_max - P_min) * 100_000 / 205 # pa / bloc
# x_density = (V_max - V_min) * 1e-6 / 21 # m^3 / bloc
# y_density = (P_max - P_min) * 100_000 / 20.3 # pascal / bloc
# x_density = (V_max - V_min) * 1e-3 / 21.1 # dm^3 / cm
# y_density = (P_max - P_min) / 20.3 # bar / cm

omega = (ufloat(229, 10) / 60) # 1 / s = Hz

callibration = (x_density * y_density) # (m^3 * Pa/bloc) = J/bloc

aire = ufloat(100*70+25*(83+5+30+29), 20) # petits bloc
# aire = ufloat(103, 5) # gros bloc
# aire = ufloat(116, 10) # cm^2

W = callibration * aire
P_m = W * omega # W = J/s
print(f"Puissance: {P_m:.1uP} W")

rho_pv = P_m / phi2
print(f"Efficacité PV: {rho_pv:.1u%}")

Puissance: 13±4 W
Efficacité PV: (7+/-3)%


## Methode freinage

In [7]:
data = load("../data/couple-regime2.csv")
data

,omega [rpm],F [N]
0,229.0,0.0
1,215.0,0.2
2,207.6,0.4
3,200.0,0.6
4,194.6,0.8
5,192.3,1.0
6,189.7,1.2
7,192.3,1.4
8,190.2,1.6
9,187.2,1.8


In [8]:
ω = unp.uarray(data["omega [rpm]"], 10) * 1/60  # tours / s
F = unp.uarray(data["F [N]"], 0.05) # N = kg * m / s^2
# https://en.wikipedia.org/wiki/Torque#Conversion_to_other_units
P_m = (ω * 2 * np.pi) * F * (r_disque * 0.01) # W = kg * m^2 / s^3


rho_frein = P_m / phi2

In [9]:
print(f"Efficacité max: {np.max(rho_frein):.1u%}")
print(f"Efficacité min: {np.min(rho_frein[1:]):.1u%}")
print(f"Efficacité moyenne: {np.mean(rho_frein[1:]):.1u%}")

Efficacité max: (0.82+/-0.05)%
Efficacité min: (0.10+/-0.03)%
Efficacité moyenne: (0.47+/-0.01)%


## Difference quantités de chaleur

In [10]:
phi1 = delta_T_eau * C_m_eau * rho_eau * (debit_eau * 0.000001 * 1/60)
print(f"Phi 1: {phi1}")

P_m = phi2 - phi1
rho_chaleur = P_m / phi2
print(f"Efficacité: {rho_chaleur:.1u%}")

Phi 1: 76.2+/-2.4
Efficacité: (56+/-1)%
